In [ ]:
!pip install langchain langchain_community openai==0.28
!pip install matplotlib pillow
!pip install -U duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 972.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 33.0 MB/s eta 0:00:00


In [ ]:
import os
import time
from bs4 import BeautifulSoup
import re
import requests
import warnings

import langchain
from langchain.llms import OpenAI
from langchain.agents import load_tools, AgentType, Tool, initialize_agent

In [ ]:
import os
import time
from bs4 import BeautifulSoup
import re
import requests
import warnings

import langchain
from langchain.llms import OpenAI
from langchain.agents import load_tools, AgentType, Tool, initialize_agent

In [ ]:
os.environ["OPENAI_API_KEY"] = "Enter Your API key here"

warnings.filterwarnings("ignore")

In [ ]:
llm=OpenAI(temperature=0,
           model_name="gpt-4o-mini")

# Tools

**Finding the ticker symbol of the stock**

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import ChatPromptTemplate

def find_ticker_symbol(customer_question):

    ticker_symbol_schema = ResponseSchema(
        name="ticker_symbol",
        description= "Extract the ticker symbol for the stock mentioned in the text. If the ticker symbol for the given stock name cannot be identified or does not exist, return 'NOTFOUND'. \Ensure that the extracted ticker symbol is accurate and corresponds directly to the stock name provided."
    )

    output_parser = StructuredOutputParser.from_response_schemas([ticker_symbol_schema])
    format_instructions = output_parser.get_format_instructions()

    review_template = f"""\
    For the following text, extract the following information:

    ticker_symbol: Extract the ticker symbol for the stock mentioned in the text.\
     If the ticker symbol for the given stock name cannot be identified or does not exist, return 'NOTFOUND'. \
     Ensure that the extracted ticker symbol is accurate and corresponds directly to the stock name provided.
    text: {{text}}

    {{format_instructions}}
    """
    prompt_template = ChatPromptTemplate.from_template(review_template)

    messages = prompt_template.format_messages(text=customer_question, format_instructions=format_instructions)

    # Invoke the LLM with the formatted messages
    response = llm.invoke(messages)

    # Parse the output from the response
    try:
        output_dict = output_parser.parse(response)
        return output_dict.get("ticker_symbol", "NOTFOUND")
    except Exception as e:
        print(f"Error processing response: {e}")
        return "NOTFOUND"

#print(find_ticker_symbol("How is the APPLE stock doing today"))


**Function for finding the basic information of the stock**

In [ ]:
import yfinance as yf

def basic_stock_info(customer_question):
  ticker_symbol = find_ticker_symbol(customer_question)
  if ticker_symbol == "NOTFOUND":
    return "Error processing stock name in the question"
  try:
    ticker = yf.Ticker(ticker_symbol)
  except Exception as e:
    return("Error processing response")

  info = ticker.info

  company_info = (
    f"Below are the Basic details of {info['longName']} ::\n"
    f"Company name: {info['longName']},\n"
    f"Current price: {info['currentPrice']},\n"
    f"Market cap: {info['marketCap']},\n"
    f"PE Ratio (Trailing): {info['trailingPE']},\n"
    f"Dividend Yield: {info['dividendYield'] * 100:.2f}%,\n"
    f"52-Week High: {info['fiftyTwoWeekHigh']} USD,\n"
    f"52-Week Low: {info['fiftyTwoWeekLow']} USD, \n"
    f"Forward PE Ratio: {info['forwardPE']},"
  )

  return company_info

#print(basic_stock_info("AAPL"))

**Function for finding any specfic information or details of the stock**

In [ ]:
import yfinance as yf
import json

def specific_stock_inforatiom(customer_question):
  ticker_symbol = find_ticker_symbol(customer_question)
  if ticker_symbol == "NOTFOUND":
    return "Error processing stock name in the question"
  try:
    ticker = yf.Ticker(ticker_symbol)
  except Exception as e:
    return("Error processing response")

  details = str([
    str(ticker.info),
    ticker.recommendations.to_string(),
    ticker.balance_sheet.to_string(),
    ticker.financials.to_string(),
    ticker.cashflow.to_string(),
    ticker.actions.to_string(),
    ticker.sustainability.to_string(),
    ticker.institutional_holders.to_string(),
    ticker.mutualfund_holders.to_string(),
    ticker.major_holders.to_string()
  ])

  return details

#print(specific_stock_inforatiom("How is APPLE looking today"))

**Function for plotting the historic stock price**

In [ ]:
import matplotlib.pyplot as plt

def generate_matplotlib_graph(customer_query, filename='graph.png'):
  ticker_symbol = find_ticker_symbol(customer_query)
  if ticker_symbol == "NOTFOUND":
    return "Error processing stock name in the question"
  try:
    ticker = yf.Ticker(ticker_symbol)
  except Exception as e:
    return("Error processing response")

  try:
    historical_data = ticker.history(period="max")
  except Exception as e:
    return "Error fetching historical data: " + str(e)

  plt.plot(historical_data.index, historical_data["Close"], linestyle='-', color='b')
  plt.ylabel("Price in USD")
  plt.title("Historic stock price")
  plt.grid(True)

  image_path = os.path.join("/content", filename)

  try:
    plt.savefig(image_path)
  except Exception as e:
    return "Error saving plot: " + str(e)
  finally:
        plt.close()

  return image_path
generate_matplotlib_graph("Apple stock")

'/content/graph.png'

**Function for stock price prediction**

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

def stock_price_prediction(customer_query):
  ticker_symbol = find_ticker_symbol(customer_query)
  if ticker_symbol == "NOTFOUND":
    return "Error processing stock name in the question"
  try:
    ticker = yf.Ticker(ticker_symbol)
    historical_data = ticker.history(period="1y")
    data = historical_data.filter(['Close'])
    dataset = data.values
    training_data_len = int(np.ceil( len(dataset) * .95 ))
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(dataset)
    train_data = scaled_data[0:int(training_data_len), :]
    x_train = []
    y_train = []
    for i in range(60, len(train_data)):

      x_train.append(train_data[i-60:i, 0])
      y_train.append(train_data[i, 0])
    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape= (x_train.shape[1], 1)))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))

    #Compile
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(x_train, y_train, batch_size=1, epochs=1, verbose = False)

    test_data = scaled_data[training_data_len - 60: , :]
    # Create the data sets x_test and y_test
    x_test = []
    y_test = dataset[training_data_len:, :]
    for i in range(60, len(test_data)):
      x_test.append(test_data[i-60:i, 0])
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))
    # Predictions
    predictions = model.predict(x_test, verbose = False)
    predictions = scaler.inverse_transform(predictions)

    rmse = np.sqrt(np.mean((predictions - y_test) ** 2))
    mean_y_test = np.mean(y_test)

    # Calculate the percentage error
    percentage_error = (rmse / mean_y_test) * 100

    result = f"The stock price is predicted with  {percentage_error:.2f}% error:"
    # train = data[:training_data_len]
    # valid = data[training_data_len:]
    # valid['Predictions'] = predictions
    # # Visualize the data
    # plt.figure(figsize=(7,5))
    # plt.title('Model')
    # plt.xlabel('Date', fontsize=18)
    # plt.ylabel('Close Price USD ($)', fontsize=18)
    # plt.plot(train['Close'])
    # plt.plot(valid[['Close', 'Predictions']])
    # plt.plot()
    # plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
    # plt.show()

    #Forecasting the price
    future_data = historical_data.filter(['Close'])
    last_90_days = future_data[-90:]
    last_90_days_scaled = scaler.transform(last_90_days)
    X_test = []
    X_test.append(last_90_days_scaled)
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    predicted_prices = []
    for i in range(10):
        predicted_price = model.predict(X_test, verbose = False)
        predicted_prices.append(predicted_price[0][0])
        X_test = np.append(X_test, [predicted_price], axis=1)

    # invert the scaling
    predicted_prices = np.array(predicted_prices).reshape(-1, 1)
    predicted_prices = scaler.inverse_transform(predicted_prices)
    print("Predicted stock price for next 10 days ::")
    for i in range(0, len(predicted_prices)):
      result = result + f"\n Day {i+1} is {predicted_prices[i][0]} "

  except Exception as e:
    return("Error processing response")

  return result
print(stock_price_prediction("Apple stock"))

Predicted stock price for next 10 days ::
The stock price is predicted with  4.23% error:
 Day 1 is 222.2845001220703 
 Day 2 is 222.1595458984375 
 Day 3 is 222.2861785888672 
 Day 4 is 222.58233642578125 
 Day 5 is 222.99215698242188 
 Day 6 is 223.47811889648438 
 Day 7 is 224.01548767089844 
 Day 8 is 224.5882568359375 
 Day 9 is 225.18614196777344 
 Day 10 is 225.80267333984375 


**Finding the agent execution tool chain**

In [ ]:
import re
def find_tools_chain(intermediate_string):
  # Define the patterns
  intermediate_string = str(intermediate_string)
  tool_pattern = re.compile(r"tool='([^']*)'")
  tool_input_pattern = re.compile(r"tool_input='([^']*)'")

  # Find the matchees
  tools = tool_pattern.findall(intermediate_string)
  tool_inputs = tool_input_pattern.findall(intermediate_string)

  tool_chain = " --> ".join(tools)

  return tool_chain
#find_tools_chain(response["intermediate_steps"])

In [ ]:
import langchain
from langchain.llms import OpenAI
from langchain.agents import load_tools, AgentType, Tool, initialize_agent

In [ ]:
llm.invoke("Hello")

'Hello! How can I assist you today?'

In [ ]:
tools = [
    Tool(
        name = "find stock ticker symbol",
        func =  find_ticker_symbol ,
        description = "Use when you want to extract the ticker symbol of the stock"

    ),
    Tool(
        name = "specific stock information",
        func = specific_stock_inforatiom ,
        description = "Use when to find the specific details or information of the stock"
    ),
    Tool(
        name = "Historic stock plot",
        func = generate_matplotlib_graph ,
        description = "Use when you want to show or display the image path of image of historic stock price "
    ),
    Tool(
        name = "Stock price prediction" ,
        func = stock_price_prediction ,
        description = "Use when you want to forecast or predict the future stock price"
    )
]


In [ ]:
from langchain.agents import initialize_agent

zero_shot_agent=initialize_agent(
    llm=llm,
    agent="zero-shot-react-description",
    tools=tools,
    verbose=False,
    max_iteration=4,
    return_intermediate_steps=True,
    handle_parsing_errors=True
)

In [ ]:
print(zero_shot_agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

find stock ticker symbol(customer_question) - Use when you want to extract the ticker symbol of the stock
specific stock information(customer_question) - Use when to find the specific details or information of the stock
Historic stock plot(customer_query, filename='graph.png') - Use when you want to show or display the image path of image of historic stock price 
Stock price prediction(customer_query) - Use when you want to forecast or predict the future stock price

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [find stock ticker symbol, specific stock information, Historic stock plot, Stock price prediction]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the fi

In [ ]:
new_prompt = """You are a financial advisor Chatbot.Keep the reponses small and concise.You are a financial advisor tasked with
advising the users with the stock information.I

Answer the following questions as best you can .You have access to the following tools:

find stock ticker symbol(customer_question) - Use when you want to extract the ticker symbol of the stock
specific stock information(customer_question) - Use when to find the information or detials  of a stock
Historic stock plot(customer_query, filename='graph.png') - Use when you want to show or display the image path of image of historic stock price
Stock price prediction(customer_query) - Use when you want to forecast or predict the future stock price


Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [find stock ticker symbol, basic stock info]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

In [ ]:
zero_shot_agent.agent.llm_chain.prompt.template=new_prompt

In [ ]:
response = zero_shot_agent("How does apple stock looks like")
print(response["output"]), print("Execution tool chain ::\n ",find_tools_chain(response["intermediate_steps"]))

Apple Inc. (AAPL) is currently priced at $219.86, with a market cap of approximately $3.34 trillion. It operates in the Consumer Electronics industry and has a dividend yield of 0.45%. The stock has a 52-week range of $164.08 to $237.23. For more details, visit [Apple's website](https://www.apple.com).
Execution tool chain ::
  find stock ticker symbol --> specific stock information


(None, None)

In [ ]:
response = zero_shot_agent("What is the ticker symbol of NVIDIA stock")
print(response["output"]), print("Execution tool chain ::\n ",find_tools_chain(response["intermediate_steps"]))

The ticker symbol for NVIDIA stock is NVDA.
Execution tool chain ::
  find stock ticker symbol


(None, None)

In [ ]:
response = zero_shot_agent("Give me the basic details of the NANDA stock")
print(response["output"]), print("Execution tool chain ::\n ",find_tools_chain(response["intermediate_steps"]))

I couldn't find any information on NANDA stock. Please check the name or provide a different stock.
Execution tool chain ::
  find stock ticker symbol


(None, None)

In [ ]:
response = zero_shot_agent("Give me last quarter  performance of the nvidia stock")
print(response["output"]), print("Execution tool chain ::\n ",find_tools_chain(response["intermediate_steps"]))

NVIDIA's last quarter performance shows a significant increase in revenue and net income, with total revenue of approximately $60.92 billion and net income of about $29.76 billion. The stock is currently trading at $107.27.
Execution tool chain ::
  find stock ticker symbol --> specific stock information


(None, None)

In [ ]:
response = zero_shot_agent("What is return on Assets of the nvidia stock")
print(response["output"]), print("Execution tool chain ::\n ",find_tools_chain(response["intermediate_steps"]))

The return on assets (ROA) for NVIDIA (NVDA) is approximately 49.1%.
Execution tool chain ::
  find stock ticker symbol --> specific stock information


(None, None)

In [ ]:
response = zero_shot_agent("Give me a stock name which i should not invest with an example and why(Show the details)")
print(response["output"]), print("Execution tool chain ::\n ",find_tools_chain(response["intermediate_steps"]))

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/GME?modules=esgScores&corsDomain=finance.yahoo.com&formatted=false&symbol=GME&crumb=s8vl76%2FamZX


A stock often advised against is GameStop (GME). It has high volatility, a trailing P/E ratio of 263.38, and significant debt, making it a risky investment.
Execution tool chain ::
  find stock ticker symbol --> specific stock information


(None, None)

In [ ]:
response = zero_shot_agent("What is the current price of the apple stock")
print(response["output"]), print("Execution tool chain ::\n ",find_tools_chain(response["intermediate_steps"]))

The current price of Apple stock (AAPL) is $219.86.
Execution tool chain ::
  find stock ticker symbol --> specific stock information


(None, None)

In [ ]:
import yfinance as yf

ticker = yf.Ticker("AAPL")
details = [
str(ticker.info),
ticker.recommendations.to_string(),
ticker.balance_sheet.to_string(),
ticker.financials.to_string(),
ticker.cashflow.to_string(),
]

In [ ]:
response = zero_shot_agent("Show me a plot of  apple stock price over years")
print(response["output"]), print("Execution tool chain ::\n ",find_tools_chain(response["intermediate_steps"]))

You can view the historic stock price plot for Apple (AAPL) [here](sandbox:/content/graph.png).
Execution tool chain ::
  find stock ticker symbol --> Historic stock plot


(None, None)

In [ ]:
# Script to scrap top5 googgle news for given company name

def google_query(search_term):
    if "news" not in search_term:
        search_term=search_term+" stock news"
    url=f"https://www.google.com/search?q={search_term}&cr=countryIN"
    url=re.sub(r"\s","+",url)
    return url

def get_recent_stock_news(company_name):
    # time.sleep(4) #To avoid rate limit error
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

    g_query=google_query(company_name)
    res=requests.get(g_query,headers=headers).text
    soup=BeautifulSoup(res,"html.parser")
    news=[]
    for n in soup.find_all("div","n0jPhd ynAwRc tNxQIb nDgy9d"):
        news.append(n.text)
    for n in soup.find_all("div","IJl0Z"):
        news.append(n.text)


    if len(news)>6:
        news=news[:4]
    else:
        news=news
    news_string=""
    for i,n in enumerate(news):
        news_string+=f"{i}. {n}\n"
    top5_news="Recent News:\n\n"+news_string

    return top5_news


print(get_recent_stock_news("apple"))

Recent News:

0. Warren Buffett’s Berkshire Hathaway slashes Apple stake by 50% on selling 
spree, cash holding nears $280 billion | Stock Market News
1. Warren Buffett's Berkshire halves Apple stake, boosts cash to $277 billion 
even as operating profit sets r



In [ ]:
from langchain.tools import DuckDuckGoSearchRun
search=DuckDuckGoSearchRun()

search("Stock news apple")

'Key Facts. Apple stock rose as much as 2.9% to a new intraday peak of $237.23 in Monday trading, with its over $234 share price by mid-afternoon on pace to top last week\'s record close of $232. ... The Goldman Sachs analyst Michael Ng also grew more upbeat on Apple stock following its results.. Ng raised his Apple price target to $275 from $265, arguing that the iPhone maker was "on the cusp ... Apple\'s stock fluctuated narrowly in after-hours trading. The stock fell 1.7% on Thursday, but was still up 13% year-to-date through the close. Advertisement. 2024-08-01T21:59:53Z Apple\'s stock surged 7% on Tuesday to reach a record-high close for the first time in 2024 as investors digested the announcement of its AI platform, Apple Intelligence.Shares in the iPhone maker ... Discover real-time Apple Inc. Common Stock (AAPL) stock prices, quotes, historical data, news, and Insights for informed trading and investment decisions. Stay ahead with Nasdaq.'